In [26]:
import sys
import os
import json

import pandas as pd
import numpy as np

from collections import defaultdict

In [27]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/JSON_to_PGSQL'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

In [28]:
raw_data_path = config.raw_data

---

In [29]:
with open(raw_data_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

---

# **Drill 1**

In [30]:
level_top = [key for key in raw_data.keys()]
level_top

['kind', 'data']

In [31]:
level_2_data = [key for key in raw_data['data'].keys()]
level_2_data

['after', 'dist', 'modhash', 'geo_filter', 'children', 'before']

In [32]:
level_2_listing = raw_data['kind']
level_2_listing

'Listing'

---

# **Drill 2**

In [33]:
level_2_options = [key for key in raw_data['data'].keys()]
level_2_options

['after', 'dist', 'modhash', 'geo_filter', 'children', 'before']

In [35]:
posts = raw_data['data']['children']
posts[0]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'WhitePeopleTwitter',
  'selftext': '',
  'author_fullname': 't2_8a90k0',
  'saved': False,
  'mod_reason_title': None,
  'gilded': 0,
  'clicked': False,
  'title': "Yeah, that's not okay.",
  'link_flair_richtext': [{'e': 'text', 't': 'Clubhouse'}],
  'subreddit_name_prefixed': 'r/WhitePeopleTwitter',
  'hidden': False,
  'pwls': None,
  'link_flair_css_class': None,
  'downs': 0,
  'thumbnail_height': 140,
  'top_awarded_type': None,
  'hide_score': False,
  'name': 't3_14a0ayt',
  'quarantine': False,
  'link_flair_text_color': 'dark',
  'upvote_ratio': 0.96,
  'author_flair_background_color': None,
  'ups': 12836,
  'total_awards_received': 0,
  'media_embed': {},
  'thumbnail_width': 140,
  'author_flair_template_id': None,
  'is_original_content': False,
  'user_reports': [],
  'secure_media': None,
  'is_reddit_media_domain': True,
  'is_meta': False,
  'category': None,
  'secure_media_embed': {},
  'link_flair_t

In [38]:
[post['data'].keys() for post in posts][0:3]

[dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'post_hint', 'content_categories', 'is_self', 'subreddit_type', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html'

In [39]:
[post['data'].values() for post in posts][0:3]

[dict_values([None, 'WhitePeopleTwitter', '', 't2_8a90k0', False, None, 0, False, "Yeah, that's not okay.", [{'e': 'text', 't': 'Clubhouse'}], 'r/WhitePeopleTwitter', False, None, None, 0, 140, None, False, 't3_14a0ayt', False, 'dark', 0.96, None, 12836, 0, {}, 140, None, False, [], None, True, False, None, {}, 'Clubhouse', False, 12836, None, False, True, 'https://b.thumbs.redditmedia.com/hv0ShtFGoPsuvfG6mqpuNCRxxO6Oabrg-HL7m0-g0ig.jpg', False, None, [], {}, 'image', None, False, 'public', 1686829569, 'richtext', None, None, None, 'text', 'i.redd.it', True, None, None, None, None, 'https://i.redd.it/urxp0hrc666b1.png', None, False, False, True, False, False, {'images': [{'source': {'url': 'https://preview.redd.it/urxp0hrc666b1.png?auto=webp&amp;v=enabled&amp;s=ae53a9e5e97d4362ebfebd77a1abdde8a7ca41fa', 'width': 1025, 'height': 1628}, 'resolutions': [{'url': 'https://preview.redd.it/urxp0hrc666b1.png?width=108&amp;crop=smart&amp;auto=webp&amp;v=enabled&amp;s=75ebf6d794e5aa01c3d4a5bf7b6

---

# **Initial Exploration**

In [40]:
single_post = raw_data['data']['children'][0]
single_post

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'WhitePeopleTwitter',
  'selftext': '',
  'author_fullname': 't2_8a90k0',
  'saved': False,
  'mod_reason_title': None,
  'gilded': 0,
  'clicked': False,
  'title': "Yeah, that's not okay.",
  'link_flair_richtext': [{'e': 'text', 't': 'Clubhouse'}],
  'subreddit_name_prefixed': 'r/WhitePeopleTwitter',
  'hidden': False,
  'pwls': None,
  'link_flair_css_class': None,
  'downs': 0,
  'thumbnail_height': 140,
  'top_awarded_type': None,
  'hide_score': False,
  'name': 't3_14a0ayt',
  'quarantine': False,
  'link_flair_text_color': 'dark',
  'upvote_ratio': 0.96,
  'author_flair_background_color': None,
  'ups': 12836,
  'total_awards_received': 0,
  'media_embed': {},
  'thumbnail_width': 140,
  'author_flair_template_id': None,
  'is_original_content': False,
  'user_reports': [],
  'secure_media': None,
  'is_reddit_media_domain': True,
  'is_meta': False,
  'category': None,
  'secure_media_embed': {},
  'link_flair_t

In [41]:
single_post_data = raw_data['data']['children'][0]['data']
single_post_data

{'approved_at_utc': None,
 'subreddit': 'WhitePeopleTwitter',
 'selftext': '',
 'author_fullname': 't2_8a90k0',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': "Yeah, that's not okay.",
 'link_flair_richtext': [{'e': 'text', 't': 'Clubhouse'}],
 'subreddit_name_prefixed': 'r/WhitePeopleTwitter',
 'hidden': False,
 'pwls': None,
 'link_flair_css_class': None,
 'downs': 0,
 'thumbnail_height': 140,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_14a0ayt',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'upvote_ratio': 0.96,
 'author_flair_background_color': None,
 'ups': 12836,
 'total_awards_received': 0,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': True,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text': 'Clubhouse',
 'can_mod_post': False,
 'score': 12836,

In [43]:
[post['data']['title'] for post in posts][0:3]

["Yeah, that's not okay.",
 'Just 1 neat single page law would completely change the housing market.',
 'To speed because he is a cop.']

In [45]:
# print the title of each post
for post in posts[0:3]:
    print(post['data']['title'])

Yeah, that's not okay.
Just 1 neat single page law would completely change the housing market.
To speed because he is a cop.


---

In [46]:
key_counts = defaultdict(int)  # initialize a default dictionary

for post in posts:
    for key in post['data'].keys():
        key_counts[key] += 1  # increment the count for each key
key_counts

defaultdict(int,
            {'approved_at_utc': 25,
             'subreddit': 25,
             'selftext': 25,
             'author_fullname': 25,
             'saved': 25,
             'mod_reason_title': 25,
             'gilded': 25,
             'clicked': 25,
             'title': 25,
             'link_flair_richtext': 25,
             'subreddit_name_prefixed': 25,
             'hidden': 25,
             'pwls': 25,
             'link_flair_css_class': 25,
             'downs': 25,
             'thumbnail_height': 25,
             'top_awarded_type': 25,
             'hide_score': 25,
             'name': 25,
             'quarantine': 25,
             'link_flair_text_color': 25,
             'upvote_ratio': 25,
             'author_flair_background_color': 25,
             'ups': 25,
             'total_awards_received': 25,
             'media_embed': 25,
             'thumbnail_width': 25,
             'author_flair_template_id': 25,
             'is_original_content': 25,


In [47]:
len(key_counts)

113